# **Machine Learning**

El propósito aquí es desarrollar un sistema de recomendación que pueda ser accesible a través de la API. Este sistema recibirá el ID de un juego y devolverá cinco juegos similares utilizando la técnica de similitud del coseno. Es posible que necesitemos preparar los datos, posiblemente creando un DataFrame compuesto de columnas de los archivos disponibles en el proceso de extracción, transformación y carga (ETL). Esto se hace para mejorar el entrenamiento del modelo y proporcionar recomendaciones más precisas.

In [1]:
# Importamos bibliotecas necesarias
import pandas as pd
import numpy as np
from numpy import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

Leemos los archivos para decidir qué columnas tendrá el DataFrame con el que entrenaremos nuestro modelo.

In [2]:
df_games = pd.read_csv("Data/df_output_steam_games")
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28785 entries, 0 to 28784
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    28785 non-null  int64  
 1   genres        28785 non-null  object 
 2   title         28785 non-null  object 
 3   specs         28785 non-null  object 
 4   price         28785 non-null  float64
 5   early_access  28785 non-null  bool   
 6   id            28785 non-null  int64  
 7   developer     28785 non-null  object 
 8   release_year  28785 non-null  float64
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 1.8+ MB


In [3]:
df_reviews = pd.read_parquet("Data/df_australian_user_reviews")
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  int32 
 2   recommend           59305 non-null  bool  
 3   review              59305 non-null  object
 4   posted_year         59305 non-null  int32 
 5   title               59305 non-null  object
 6   developer           59305 non-null  object
 7   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int32(2), int64(1), object(4)
memory usage: 2.8+ MB


In [4]:
df_items = pd.read_parquet("Data/df_australian_users_items")
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4207803 entries, 0 to 5153207
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   item_name         object 
 2   playtime_forever  int64  
 3   playtime_2weeks   int64  
 4   steam_id          object 
 5   items_count       int32  
 6   user_id           object 
 7   genres            object 
 8   price             float64
 9   developer         object 
 10  release_year      int32  
dtypes: float64(1), int32(3), int64(2), object(5)
memory usage: 337.1+ MB


In [5]:
# Hacemos un merged con los dataframes
merged_df = pd.merge(df_items, df_reviews[['user_id', 'posted_year', 'recommend', 'sentiment_analysis']], on='user_id', how='inner')
merged_df

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
1,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
2,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
3,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,[Action],4.99,Valve,1999,2011,True,2
4,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,[Action],4.99,Valve,1999,2011,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430161,422970,Devil Daggers,0,0,76561198312638244,36,76561198312638244,"[Action, Indie]",4.99,Sorath,2016,2014,True,2
6430162,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
6430163,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
6430164,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2


In [6]:
# Eliminamos columnas innecesarias
merged_df = merged_df.drop(columns=["playtime_2weeks", "steam_id"])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6430166 entries, 0 to 6430165
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   item_id             int32  
 1   item_name           object 
 2   playtime_forever    int64  
 3   items_count         int32  
 4   user_id             object 
 5   genres              object 
 6   price               float64
 7   developer           object 
 8   release_year        int32  
 9   posted_year         int32  
 10  recommend           bool   
 11  sentiment_analysis  int64  
dtypes: bool(1), float64(1), int32(4), int64(2), object(4)
memory usage: 447.7+ MB


In [7]:
# Definimos una variable de tipo lista con las columnas que tomaremos para el modelo
datos_agrupados = ["item_id", "item_name", "playtime_forever", "items_count", "user_id", "genres", "price", "developer", "release_year", "posted_year", "recommend", "sentiment_analysis"]
filtrado = merged_df[datos_agrupados]
filtrado

,item_id,item_name,playtime_forever,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis
0,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
1,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
2,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
3,20,Team Fortress Classic,0,277,76561197970982479,[Action],4.99,Valve,1999,2011,True,2
4,20,Team Fortress Classic,0,277,76561197970982479,[Action],4.99,Valve,1999,2011,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...
6430161,422970,Devil Daggers,0,36,76561198312638244,"[Action, Indie]",4.99,Sorath,2016,2014,True,2
6430162,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
6430163,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
6430164,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2


In [8]:
# Cambiamos los tipos de datos de las columnas a str
filtrado[datos_agrupados] = filtrado[datos_agrupados].astype(str)
filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6430166 entries, 0 to 6430165
Data columns (total 12 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   item_id             object
 1   item_name           object
 2   playtime_forever    object
 3   items_count         object
 4   user_id             object
 5   genres              object
 6   price               object
 7   developer           object
 8   release_year        object
 9   posted_year         object
 10  recommend           object
 11  sentiment_analysis  object
dtypes: object(12)
memory usage: 588.7+ MB


In [9]:
filtrado = filtrado.drop_duplicates()
filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4066414 entries, 0 to 6430162
Data columns (total 12 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   item_id             object
 1   item_name           object
 2   playtime_forever    object
 3   items_count         object
 4   user_id             object
 5   genres              object
 6   price               object
 7   developer           object
 8   release_year        object
 9   posted_year         object
 10  recommend           object
 11  sentiment_analysis  object
dtypes: object(12)
memory usage: 403.3+ MB


In [10]:
# Esta función la habíamos utilizado en el análisis inicial
def info_nulos(df, decimales=2):
    df_nulos = pd.DataFrame({
        "columna": df.columns,
        "Número de nulos": df.isnull().sum(),
        "Porcentaje de nulos": (df.isnull().sum() / df.shape[0]) * 100.0
    })
    df_nulos["Porcentaje de nulos"] = df_nulos["Porcentaje de nulos"].round(decimales).astype(str) + "%"
    return df_nulos

info_nulos(filtrado)

,columna,Número de nulos,Porcentaje de nulos
item_id,item_id,0,0.0%
item_name,item_name,0,0.0%
playtime_forever,playtime_forever,0,0.0%
items_count,items_count,0,0.0%
user_id,user_id,0,0.0%
genres,genres,0,0.0%
price,price,0,0.0%
developer,developer,0,0.0%
release_year,release_year,0,0.0%
posted_year,posted_year,0,0.0%


In [11]:
# Definimos una función para mezclar columnas en una
def mezclar_en_columna(df):
    return df["user_id"]+ " "+ df["item_name"]+ " " + df["items_count"]+ " " + df["playtime_forever"]+ " " + df["developer"]+ " " + df["genres"]+ " "+ df["price"]+ " "+ df["release_year"]+ " "+ df["posted_year"]+ " "+ df["item_id"]+ " "+ df["recommend"]+ " "+ df["sentiment_analysis"]

In [12]:
# Creamos una columna llamada "datos_agrupados" que integra todas las demas
filtrado["datos_agrupados"]=filtrado.apply(mezclar_en_columna, axis=1)
filtrado

,item_id,item_name,playtime_forever,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis,datos_agrupados
0,10,Counter-Strike,6,277,76561197970982479,['Action'],9.99,Valve,2000,2011,True,2,76561197970982479 Counter-Strike 277 6 Valve [...
3,20,Team Fortress Classic,0,277,76561197970982479,['Action'],4.99,Valve,1999,2011,True,2,76561197970982479 Team Fortress Classic 277 0 ...
6,30,Day of Defeat,7,277,76561197970982479,['Action'],4.99,Valve,2003,2011,True,2,76561197970982479 Day of Defeat 277 7 Valve ['...
9,40,Deathmatch Classic,0,277,76561197970982479,['Action'],4.99,Valve,2001,2011,True,2,76561197970982479 Deathmatch Classic 277 0 Val...
12,50,Half-Life: Opposing Force,0,277,76561197970982479,['Action'],4.99,Gearbox Software,1999,2011,True,2,76561197970982479 Half-Life: Opposing Force 27...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430146,730,Counter-Strike: Global Offensive,23,36,76561198312638244,['Action'],14.99,Valve,2012,2014,True,2,76561198312638244 Counter-Strike: Global Offen...
6430150,362890,Black Mesa,2184,36,76561198312638244,['Action' 'Adventure' 'Indie' 'Early Access'],19.99,Crowbar Collective,2015,2014,True,2,76561198312638244 Black Mesa 36 2184 Crowbar C...
6430154,400040,ShareX,1,36,76561198312638244,['Audio Production' 'Design &amp; Illustration...,4.99,ShareX Team,2007,2014,True,2,76561198312638244 ShareX 36 1 ShareX Team ['Au...
6430158,422970,Devil Daggers,0,36,76561198312638244,['Action' 'Indie'],4.99,Sorath,2016,2014,True,2,76561198312638244 Devil Daggers 36 0 Sorath ['...


In [13]:
# Creamos el modelo de vectorización y la matriz
modelo = TfidfVectorizer(stop_words='english')
matriz = modelo.fit_transform(filtrado[datos_agrupados])

In [14]:
modelo

TfidfVectorizer(stop_words='english')

In [15]:
matriz

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [16]:
# Calculamos la similitud del coseno tomando todas las filas de la matriz
similitud_del_coseno = cosine_similarity(X = matriz, Y = matriz)
similitud_del_coseno.shape

(12, 12)

In [17]:
df_similitud_del_coseno = pd.DataFrame(similitud_del_coseno)
df_similitud_del_coseno

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
# Guardamos el DataFrame con la matriz de similitud en un parquet
df_similitud_del_coseno.to_parquet("Data/similitud_del_coseno")

In [19]:
# Reiniciamos el indice del dataframe y creamos una serie de índices
filtrado = filtrado.reset_index()
indice = pd.Series(filtrado.index, index=filtrado["item_id"])

In [20]:
# Revisamos las dimensiones
filtrado.shape

(4066414, 14)

In [21]:
# Creamos una función que elimine registros aleatorios de un DataFrame
# Esto con el fin de reducir el peso del archivo que generaremos para la consulta
def eliminar_registros_aleatorios(df, n):
    if n > len(df):
        raise ValueError("El número de registros a eliminar es mayor que el número de registros del DataFrame.")

    indices_aleatorios = np.random.choice(len(df), n, replace=False)
    df = df.drop(indices_aleatorios)

    return df

In [22]:
filtrado = eliminar_registros_aleatorios(filtrado, 2000000)

In [23]:
# Volvemos a revisar las dimensiones del DataFrame
filtrado.shape

(2066414, 14)

In [24]:
# Guardamos los indices a consultar y el DataFrame "filtrado"
indice.to_csv("Data/indices_modelo")
filtrado.to_parquet("Data/filtrado_modelo")